# Clustering of Toronto Neighborhoods including nearby venues

Importing required libraries

In [1]:
#importing required libraries
import numpy as np 

import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium # map rendering library

#import requests
import requests

#import geolocator
from geopy.geocoders import Nominatim

### 1. Import the csv file saved in part 2 of the assignment and creating a dataframe

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
merged_df = pd.read_csv(body)
merged_df.head()

,Postal code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889


In [4]:
merged_df.shape

(103, 5)

### 2. Getting only Boroughs containing Toronto 

In [5]:
toronto_location = merged_df[merged_df['Borough'].str.contains(r'Toronto', case=False)]
toronto_location.reset_index(drop=True, inplace=True)
toronto_location.shape

(39, 5)

In [6]:
toronto_location.head()

,Postal code,Borough,Neighborhood,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


### 3. Getting Venues under Neighborhood using Foursquare

In [7]:
CLIENT_ID = 'KG5DJJA13V4WK3ITOTRFPP3PTNNGUE12DROCY2K4O4FHXVSO' 
CLIENT_SECRET = 'GB03ZXV5E2YJAQLEUB20HFNKBRDDBBQXYD5V1Y2U44IL5DC5' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KG5DJJA13V4WK3ITOTRFPP3PTNNGUE12DROCY2K4O4FHXVSO
CLIENT_SECRET:GB03ZXV5E2YJAQLEUB20HFNKBRDDBBQXYD5V1Y2U44IL5DC5


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    limit = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
           
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues =  getNearbyVenues(names=toronto_location['Neighborhood'],
                                   latitudes=toronto_location['latitude'],
                                   longitudes=toronto_location['longitude']
                                  )

toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park , Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park , Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park , Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [10]:
toronto_venues.shape

(1611, 7)

### 4. One Hot encoding of Venue Categories

In [11]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

toronto_onehot.shape

(1611, 218)

In [12]:
fixed_columns = [toronto_onehot.columns[152]] + list(toronto_onehot.columns[:152])+list(toronto_onehot.columns[153:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(2)

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
toronto_onehot.shape

(1611, 218)

### 5. Grouping of Onehot table by Neighborhood value

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(2)

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0,0.011236,0.022472,0.000000,0.0,0.0,0.011236,0.0,...,0.0,0.0,0.011236,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.025641,0.0,0.000000,0.025641,0.025641,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
toronto_grouped.shape

(39, 218)

### 6. Finding Top 10 places in Neighborhood and creating a dataframe

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(2)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Japanese Restaurant,Cocktail Bar,Sporting Goods Shop
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,French Restaurant,Pet Store,Brewery,Sandwich Place,Hawaiian Restaurant


### 7. Clustering of Neighborhoods

In [18]:
# set number of clusters
no_clusters = 5

toronto_clustering = toronto_grouped.drop('Neighborhood', 1)

X = toronto_clustering[toronto_clustering.columns].values

In [19]:
# run k-means clustering
kmeans = KMeans(n_clusters=no_clusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### 8. Create the final ourput Dataframe displaying the Neighborhoods, cluster labels and the top 10 venue categories.

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_output = toronto_location

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_output = toronto_output.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_output.head(2) # check the last columns!

,Postal code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Restaurant,Yoga Studio,Spa,Bakery,Beer Store,Distribution Center,Electronics Store,Event Space
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889,0,Gym,Italian Restaurant,Park,Electronics Store,Bubble Tea Shop,Burger Joint,Burrito Place,Nightclub,Café,Ramen Restaurant


In [21]:
toronto_output.shape

(39, 16)

### 9. Create a Folium map and plot the neighborhood clusters in different colors.

In [22]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(no_clusters)
ys = [i + x + (i*x)**2 for i in range(no_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_output['latitude'], toronto_output['longitude'], toronto_output['Neighborhood'], toronto_output['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters